# Making a Mega File of Data 88E Materials

Concatenate the markdown files for the lecture slides, lecture notebooks, and textbook chapters into three large markdown files.

In [8]:
import os
from pathlib import Path

In [9]:
# Define your folders
# Define your folders and their summary files
folders = {
    "slides": {
        "path": Path("~/Documents/Data88E-ForTraining/F24LS_md").expanduser(),
        "summary": Path("~/Documents/Data88E-ForTraining/F24LS_md/summary.yaml").expanduser()
    },
    "lectures": {
        "path": Path("~/Documents/Data88E-ForTraining/F24Lec_MD").expanduser(),
        "summary": Path("~/Documents/Data88E-ForTraining/F24Lec_MD/LecNB_summary.yaml").expanduser()
    },
    "textbook": {
        "path": Path("~/Documents/Data88E-ForTraining/F24Textbook_MD").expanduser(),
        "summary": Path("~/Documents/Data88E-ForTraining/F24Textbook_MD/summary.yaml").expanduser()
    }
}

In [10]:
# Course-level summary (optional global context)
course_summary = Path("~/Documents/Data88E-ForTraining/course_summary.yaml").expanduser()

# Output directory
output_dir = Path("~/Documents/Data88E-ForTraining/mega_files").expanduser()
output_dir.mkdir(parents=True, exist_ok=True)

In [13]:


def make_section_mega_file(label: str, folder: Path, summary_file: Path):
    """Concatenate section summary + all .md files (recursively)."""
    out_path = output_dir / f"{label}_mega.md"
    with open(out_path, "w", encoding="utf-8") as outfile:
        # Insert section summary at top
        if summary_file.exists():
            outfile.write(f"\n\n--- SUMMARY for {label.upper()} ---\n\n")
            outfile.write(summary_file.read_text(encoding="utf-8"))
            outfile.write("\n\n")

        # Insert all markdown files
        for md_file in sorted(folder.rglob("*.md")):
            outfile.write(f"\n\n--- START {md_file.relative_to(folder)} ---\n\n")
            outfile.write(md_file.read_text(encoding="utf-8"))
            outfile.write(f"\n\n--- END {md_file.relative_to(folder)} ---\n\n")

    print(f"✅ Created {out_path}")

def make_course_summary_file():
    """Create a standalone course summary mega-file."""
    if course_summary.exists():
        out_path = output_dir / "course_mega.md"
        with open(out_path, "w", encoding="utf-8") as outfile:
            outfile.write(f"--- COURSE SUMMARY ---\n\n")
            outfile.write(course_summary.read_text(encoding="utf-8"))
        print(f"✅ Created {out_path}")
    else:
        print("⚠️ No course summary found")



In [14]:
# Build mega files for each folder
for label, meta in folders.items():
    make_section_mega_file(label, meta["path"], meta["summary"])

# Build standalone course summary file
make_course_summary_file()

✅ Created /Users/ericvandusen/Documents/Data88E-ForTraining/mega_files/slides_mega.md
✅ Created /Users/ericvandusen/Documents/Data88E-ForTraining/mega_files/lectures_mega.md
✅ Created /Users/ericvandusen/Documents/Data88E-ForTraining/mega_files/textbook_mega.md
✅ Created /Users/ericvandusen/Documents/Data88E-ForTraining/mega_files/course_mega.md
